# 🧠 AI Agents Bootcamp: Running DeepSeek with CrewAI and Ollama
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vipbasil/aibootcamp/blob/main/Day%20I/crew_ollama_deepseek.ipynb?raw=true)

This notebook is part of the AI Agents Bootcamp (23–27 June 2025) — it shows how to:
- Set up the `Ollama` environment in Colab
- Run `DeepSeek` models locally for use in CrewAI/MAS pipelines
- Prepare agents that use locally-hosted LLMs with memory and tools

## ⚙️ Step 1: Environment Setup
This installs and runs the `ollama` backend and exposes the service via localtunnel tunnel. Make sure to:
- Restart the runtime if needed
- Use the ngrok alternative (if Cloudflare is blocked or throttled)

In [ ]:
%pip install ollama
%pip install colab-xterm

## 🛠️ System Info Tools (Optional)

Installs utilities (`pciutils`, `lshw`) to inspect hardware specs — useful for checking GPU/CPU availability in Colab or local runtime.








In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

## 📦 Ollama Installation

Downloads and installs Ollama via the official shell script — run this once per environment setup.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

## 🔧 Step 2: Programmatic Model Management and Server Initialization

In this section, we:
- Import the required libraries for managing subprocesses, HTTP requests, and multithreading
- Start the Ollama server programmatically using a background thread
- Pull the required models (`deepseek-r1:7b`, `llama3`) using `ollama pull`
- Optionally include fallback to a smaller model (`deepseek-r1:1.5b`)
- Confirm the list of available models and test that the local Ollama server is running at `localhost:11434`

📌 **Why it matters**: This sets up your local model infrastructure for agent interaction. You'll later reference `localhost:11434` in your agent definitions to connect to these models.


In [ ]:
# Import necessary libraries
import subprocess
import requests
import json
import threading
from pprint import pprint

##  Launching the Ollama Server in Background

Before using any model, we need to start the **Ollama inference server**, which listens by default on `localhost:11434`.

This snippet:
- Defines a Python function `run_ollama()` that launches `ollama serve`
- Starts it in a **background thread**, so the notebook remains interactive
- Allows the server to stay active without blocking further cells

🛠️ **Note**: You only need to run this once per session. If you restart your Colab, re-run this cell before using any models.


In [ ]:
# Start the Ollama server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

## 📥 Pulling Models

We download pre-trained models from the Ollama registry:
- `deepseek-r1:7b` – reasoning & code
- `llama3` – general-purpose assistant

In [ ]:
# Download the deepseek-r1:7b distilled model
!ollama pull deepseek-r1:7b
!ollama pull llama3
# If this doesn't work, you can uncomment the below code to download a smaller model- deepseek-r1:1.5b
# !ollama pull deepseek-r1:1.5b

## 🪶 Pulling Lightweight SLMs

These small models are ideal for fast local agents and low-resource environments:
- `phi3:mini`, `tinyllama` – ultra-small general models
- `gemma:2b` – Google's compact chat model
- `deepseek-r1:1.5b` – distilled reasoning model

In [ ]:
!ollama pull phi3:mini
!ollama pull tinyllama
!ollama pull gemma:2b
!ollama pull deepseek-r1:1.5b

## 🔌 Test Ollama Server

Sends a test request to verify the Ollama server is running on `localhost:11434`.

In [ ]:
!curl http://127.0.0.1:11434

## 📄 Check Installed Models

Lists all models currently downloaded and available in your local Ollama environment.

In [ ]:
!ollama list

# 🧠 Starting the CrewAI Section

##Now we define agents using CrewAI, connected to our locally running Ollama models.  
##This enables multi-agent workflows powered by lightweight, self-hosted LLMs.


In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere

In [ ]:
%pip install -qU langchain-ollama

## 🧩 Step 3: CrewAI Integration


In [ ]:
# imports


import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from langchain_ollama import ChatOllama


## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

In [ ]:
# @title 🕵🏻 Define your agents

from crewai import Agent
from textwrap import dedent
from langchain_ollama import ChatOllama
# Define LLM (OpenAI used here; replace as needed)
from crewai import LLM

llm = LLM(model="ollama/tinyllama:latest", base_url="http://127.0.0.1:11434")



# Agent 1: Candidate Profiler
agent_1 = Agent(
    role=dedent("""Candidate Profiler"""),
    goal=dedent("""Create a clear and structured professional portrait of the candidate based on the input data."""),
    backstory=dedent("""You're a career analyst who knows how to form clear and concise professional profiles from brief descriptions."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 2: Candidate Matcher
agent_2 = Agent(
    role=dedent("""Candidate Matcher"""),
    goal=dedent("""Find suitable jobs from a list based on candidate profile including stack, budget and experience."""),
    backstory=dedent("""You are a job matching expert. Your task is to match the candidate with the most suitable position from the available list."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 3: Offer Writer
agent_3 = Agent(
    role=dedent("""Offer Writer"""),
    goal=dedent("""Compose a polite and persuasive offer letter to the candidate based on the selected vacancy."""),
    backstory=dedent("""You're an HR manager who is great at formulating letters, highlighting benefits and sounding professional but friendly."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# Agent 4: Email Sender
agent_4 = Agent(
    role=dedent("""Email Sender"""),
    goal=dedent("""Send an offer to the candidate via email (virtually)."""),
    backstory=dedent("""You're an email bot that receives the email and address and is responsible for delivering the email with a send confirmation."""),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

candidate_db = [
    {
        "id": "c-101",
        "full_name": "Alex Ivanov",
        "email": "alex@devmail.com",
        "role": "Full-Stack Developer",
        "experience_years": 5,
        "budget": 4000,
        "stack": ["React", "Node.js", "PostgreSQL", "Docker"]
    },
    {
        "id": "c-202",
        "full_name": "Maria Petrova",
        "email": "maria@devmail.com",
        "role": "Backend Developer",
        "experience_years": 4,
        "budget": 3500,
        "stack": ["Node.js", "MongoDB", "Docker", "AWS"]
    },
    {
        "id": "c-303",
        "full_name": "Oleg Smirnov",
        "email": "oleg.smirnov@devmail.com",
        "role": "DevOps Engineer",
        "experience_years": 6,
        "budget": 4500,
        "stack": ["AWS", "Docker", "Kubernetes", "Terraform", "Ansible"]
    },
    {
        "id": "c-404",
        "full_name": "Elena Kozlova",
        "email": "elena.kozlova@devmail.com",
        "role": "Frontend Developer",
        "experience_years": 3,
        "budget": 3200,
        "stack": ["Vue.js", "TypeScript", "TailwindCSS", "Vite"]
    },
    {
        "id": "c-505",
        "full_name": "Dmitry Lebedev",
        "email": "dmitry.lebedev@devmail.com",
        "role": "Data Engineer",
        "experience_years": 5,
        "budget": 4800,
        "stack": ["Python", "Spark", "Airflow", "Kafka", "Redshift"]
    },
    {
        "id": "c-606",
        "full_name": "Anna Baranov",
        "email": "anna.baranov@devmail.com",
        "role": "QA Automation Engineer",
        "experience_years": 4,
        "budget": 3300,
        "stack": ["Java", "Selenium", "Cypress", "JUnit", "Docker"]
    }
]



## Define Tasks
Tasks in CrewAI are specific assignments given to agents, detailing the actions they need to perform to achieve a particular goal. Tasks can have dependencies and context, and can be executed asynchronously to ensure an efficient workflow.

In [ ]:
# @title 📝 Define your tasks
from crewai import Task
from textwrap import dedent

print("👋 Добро пожаловать в Recruitment-Crew")
role_inp      = input("🧑‍💻 Роль кандидата?\n>> ")
exp_inp       = input("📈 Опыт кандидата (лет)?\n>> ")
budget_inp    = input("💰 Желаемая зарплата?\n>> ")
stack_inp_raw = input("🛠 Стек (через запятую)?\n>> ")

candidate_input = {
    "role": role_inp.strip(),
    "experience_years": int(exp_inp),
    "budget": int(budget_inp),
    "stack": [s.strip() for s in stack_inp_raw.split(",")]
}

print(f"✔️ Input received:\nrole: {role_inp}\nexperience_years: {exp_inp}\nbudget: {budget_inp}\nstack: {stack_inp_raw}")

# Task 1: Candidate Profiler
task_profile = Task(
    description=dedent(f"""
        Using the input {candidate_input},
        create a JSON profile of the candidate with the fields:
        id (tmp-id), role, experience_years, budget, stack (list).
    """),
    expected_output="JSON object candidate_profile",
    agent=agent_1,
    output_key="candidate_profile"
)

# Task 2: Candidate Matcher
task_match = Task(
    description=dedent("""
        Based on candidate_profile, find
        in candidate_db the candidate that matches as much as possible:
        - role match; - match or stack overlap;
        - candidate's budget ≤ required;
        - experience ≥ required.
        Return JSON matched_candidate (fields are the same as in the database)
        + match_reason field (string).
    """),
    expected_output="JSON matched_candidate",
    agent=agent_2,
    context=[task_profile],
    input_data={"candidate_db": candidate_db},
    output_key="matched_candidate"
)

# Task 3: Offer Writer
task_offer = Task(
    description=dedent("""
        Write an e-mail to the actual addressee matched_candidate.
        Markdown format, 200-250 words.
        Structure:
          - greeting by name;
          - a brief description of the position and the reasons for the match;
          - terms: stack, salary, work format;
          - call-to-action (answer, schedule a call).
    """),
    expected_output="String offer_email",
    agent=agent_3,
    context=[task_profile, task_match],
    output_key="offer_email"
)

# Task 4: Email Sender
task_send = Task(
    description=dedent("""
        Simulate sending the offer_email to email matched_candidate.
        Return the line: 'Offer sent to {full_name} <{email}> | status: success'.
    """),
    expected_output="String send_status",
    agent=agent_4,
    context=[task_offer, task_match],
    output_key="send_status"
)


In [ ]:
from crewai import Process  # если ещё нет

def main():
    crew = Crew(
        agents=[agent_1, agent_2, agent_3, agent_4],
        tasks=[task_profile, task_match, task_offer, task_send],
        process=Process.sequential,
        planning_llm=llm,
        verbose=True
    )

    crew.kickoff()  # результат в Task-ах

    # 1️⃣ профиль
    profile_raw = task_profile.output.raw

    # 2️⃣ подобранный кандидат
    match_raw = task_match.output.raw

    # 3️⃣ письмо-оффер
    offer_md = task_offer.output.raw

    # 4️⃣ подтверждение отправки
    send_line = task_send.output.raw

    print("\n👤 Подходящий кандидат:\n", match_raw)
    print("\n📤 Статус отправки:\n", send_line)

    # если нужно что-то вернуть из main():
    return {"candidate": match_raw, "status": send_line}

if __name__ == "__main__":
    result = main()

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = final_status.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))